In [1]:
!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
import time
from PIL import Image


#-------------------------------------------------------------------------------------JUPYTER NOTEBOOK SETTINGS-------------------------------------------------------------------------------------
from IPython.core.display import display, HTML                                    
display(HTML("<style>.container { width:100% !important; }</style>"))     

Defaulting to user installation because normal site-packages is not writeable


C:\Users\Ciprian-Florin Ifrim\AppData\Local\Temp\ipykernel_8432\3736244167.py:14: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
webpage = "https://binance.com/en/price"

driver = webdriver.Chrome()
driver.implicitly_wait(2) # seconds
driver.get(webpage) 

page_selector_element = "button.css-hlqxzb"
currency_name_element = "div.css-1r69m1k"

currencies_names = list()
pages = driver.find_elements(By.CSS_SELECTOR, page_selector_element)
#print(int(pages[len(pages)-1].text))

for i in range(46, 49):
    # go through all the page numbers
    driver.get(webpage + "?page=" + str(i+1)) 
    
    # find all elements with the class name requested
    try:
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, currency_name_element)))
        currencies_available = driver.find_elements(By.CSS_SELECTOR, currency_name_element)
    except TimeoutException:
        continue
    
    # check if any elements were found
    if not currencies_available:
        print("No elements found with class name " + currency_name_element)
    else:
        for element in currencies_available:
            currencies_names.append(element.text.lower().replace(" ", "-"))     
# print(currencies_names)

cookies_flag = 0
for k in range(len(currencies_names)):
    driver.get(webpage + "/" + currencies_names[k])
    if cookies_flag == 0:                                            # binance requires the cookies popup to be cleared first before interacting with the website
        try:
            time.sleep(1)
            cookies_btn_xpath = "//div[@id='onetrust-button-group']//button[@id='onetrust-reject-all-handler']"
            WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, cookies_btn_xpath))).click()
            cookies_flag = 1
        except TimeoutException:
            continue
    
    try:
        WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.CSS_SELECTOR, "button.css-kdgs4h")))
        timeframe_buttons = driver.find_elements(By.CSS_SELECTOR, "button.css-kdgs4h")
        #print(timeframe_buttons)
    except TimeoutException:
        continue
    
    loop_counter, refresh_flag = 0, 0
    print("Size:", len(timeframe_buttons))
    while loop_counter < len(timeframe_buttons):
        # If a StaleElementReferenceException occurs, re-locate the element
        try:
            timeframe_buttons[loop_counter].click()
        except StaleElementReferenceException:
            WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.CSS_SELECTOR, "button.css-kdgs4h")))
            timeframe_buttons = driver.find_elements(By.CSS_SELECTOR, "button.css-kdgs4h")
            timeframe_buttons[loop_counter].click()
        time.sleep(2)
        
        missing_data_element = driver.find_elements(By.XPATH, '//p[@class="title" and text()="Couldn’t find any data"]')
        if not missing_data_element: 
            print("Data is present")
            pass
        else: 
            print("Data is missing. Skipping currency!")
            if refresh_flag < 2:
                refresh_flag, loop_counter = refresh_flag + 1, 0
                driver.refresh()
                time.sleep(1)
                continue
            else:
                break
        
        try:
            refresh_elm_xpath = '//button[@style="margin-top: 32px; background-color: var(--theme-color);" and text()="Refresh"]'
            refresh_plot_elm = WebDriverWait(driver,1).until(EC.element_to_be_clickable((By.XPATH, refresh_elm_xpath))).click()
            loop_counter = 0
            continue
        except TimeoutException:
            pass
        else:
            loop_counter += 1
            continue

        # get the dimensions of the graph element
        graph = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.chart")))
        graph_location = graph.location
        graph_size = graph.size

        currency_name = driver.find_element(By.CSS_SELECTOR, "div.css-9uk0ol")
        currency_name_size = currency_name.size
        print(currency_name_size)

        volatility_warning = None
        listing_warning = None

        try: 
            volatility_warning = driver.find_element(By.CSS_SELECTOR, "div.css-2tlblt")
            volatility_warning_size = volatility_warning.size
            print(volatility_warning_size)
        except NoSuchElementException:
            pass

        try: 
            listing_warning = driver.find_element(By.CSS_SELECTOR, "div.css-17us7ga")
            listing_warning_size = listing_warning.size
            print(listing_warning_size)
        except NoSuchElementException:
            pass

        graph_top    =  graph_location['y'] - 110 - currency_name_size['height']
        graph_left   =  graph_location['x']
        graph_right  =  graph_location['x'] + graph.size['width'] + 15
        graph_bottom =  graph_location['y'] + graph.size['height']

        screenshot_folder_name = "/no_warnings/"

        if volatility_warning and listing_warning:
            graph_top = graph_top - volatility_warning_size['height'] - listing_warning_size['height']    # adjusted by both warning sizes to include the bitcoin title name
            screenshot_folder_name = "/both_warnings/"
        elif volatility_warning and not listing_warning:
            graph_top = graph_top - volatility_warning_size['height']                                     # adjusted by warning size to include the bitcoin title name
            screenshot_folder_name = "/volatility_warning/"
        elif listing_warning and not volatility_warning:
            graph_top = graph_top - listing_warning_size['height']                                         # adjusted by warning size to include the bitcoin title name
            screenshot_folder_name = "/listing_warning/"

            
        # take screenshot of the page
        filepath = "sample_plots/" + webpage.split("/")[2] + screenshot_folder_name 
        screenshot_name = timeframe_buttons[loop_counter].text + "_" + currencies_names[k] + ".png"
        driver.save_screenshot(filepath + screenshot_name)
        
        # open the screenshot and crop it to the dimensions of the element
        image = Image.open(filepath + screenshot_name)
        
        image = image.crop((graph_left, graph_top, graph_right, graph_bottom))
        image.save(filepath + screenshot_name)

        print("Run " + str(loop_counter) + ":", timeframe_buttons[loop_counter].text, currencies_names[k])
        loop_counter += 1

driver.quit()

The version of chrome cannot be detected. Trying with latest driver version


Size: 5
Data is present
{'height': 57, 'width': 768}
{'height': 72, 'width': 768}
Run 0: 7D crypto-rangers
Data is present
{'height': 57, 'width': 768}
{'height': 72, 'width': 768}
Run 1: 1M crypto-rangers
Data is present
{'height': 57, 'width': 768}
{'height': 72, 'width': 768}
Run 2: 3M crypto-rangers
Data is present
{'height': 57, 'width': 768}
{'height': 72, 'width': 768}
Run 3: 1Y crypto-rangers
Data is present
{'height': 57, 'width': 768}
{'height': 72, 'width': 768}
Run 4: YTD crypto-rangers
Size: 5
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 0: 7D zodium
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 1: 1M zodium
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 2: 3M zodium
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 3: 1Y zodium
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 4: YTD zodium
Size: 5
Data is missing. Skippin

Run 3: 1Y twitfi
Data is present
{'height': 57, 'width': 768}
{'height': 40, 'width': 712}
{'height': 48, 'width': 768}
Run 4: YTD twitfi
Size: 5
Data is missing. Skipping currency!
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 0: 7D baskonia-fan-token
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 1: 1M baskonia-fan-token
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 2: 3M baskonia-fan-token
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 3: 1Y baskonia-fan-token
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 4: YTD baskonia-fan-token
Size: 5
Data is missing. Skipping currency!
Data is present
{'height': 57, 'width': 768}
{'height': 40, 'width': 712}
{'height': 48, 'width': 768}
Run 0: 7D read2n
Data is present
{'height': 57, 'width': 768}
{'height': 40, 'width': 712}
{'height': 48, 'width': 768}
Run 1: 1M read2n
Data is present
{'

Data is missing. Skipping currency!
Size: 5
Data is missing. Skipping currency!
Data is present
{'height': 57, 'width': 768}
{'height': 40, 'width': 712}
{'height': 48, 'width': 768}
Run 0: 7D uniwhale
Data is present
{'height': 57, 'width': 768}
{'height': 40, 'width': 712}
{'height': 48, 'width': 768}
Run 1: 1M uniwhale
Data is present
{'height': 57, 'width': 768}
{'height': 40, 'width': 712}
{'height': 48, 'width': 768}
Run 2: 3M uniwhale
Data is present
{'height': 57, 'width': 768}
{'height': 40, 'width': 712}
{'height': 48, 'width': 768}
Run 3: 1Y uniwhale
Data is present
{'height': 57, 'width': 768}
{'height': 40, 'width': 712}
{'height': 48, 'width': 768}
Run 4: YTD uniwhale
Size: 5
Data is present
{'height': 57, 'width': 768}
{'height': 40, 'width': 712}
{'height': 48, 'width': 768}
Run 0: 7D future1coin
Data is present
{'height': 57, 'width': 768}
{'height': 40, 'width': 712}
{'height': 48, 'width': 768}
Run 1: 1M future1coin
Data is present
{'height': 57, 'width': 768}
{'heig

Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 2: 3M celebplus
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 3: 1Y celebplus
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 4: YTD celebplus
Size: 5
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 0: 7D shitgpt
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 1: 1M shitgpt
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 2: 3M shitgpt
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 3: 1Y shitgpt
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 4: YTD shitgpt
Size: 5
Data is missing. Skipping currency!
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 0: 7D bnbpot
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 1: 1M bnbpot
Data is present
{'heig

...

Data is present
{'height': 57, 'width': 768}
{'height': 80, 'width': 712}
{'height': 48, 'width': 768}
Run 3: 1Y masterpiece-maker
Data is present
{'height': 57, 'width': 768}
{'height': 80, 'width': 712}
{'height': 48, 'width': 768}
Run 4: YTD masterpiece-maker
Size: 5
Data is missing. Skipping currency!
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 0: 7D playpad
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 1: 1M playpad
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 2: 3M playpad
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 3: 1Y playpad
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 4: YTD playpad
Size: 5
Data is missing. Skipping currency!
Data is present
{'height': 57, 'width': 768}
{'height': 72, 'width': 768}
Run 0: 7D eagle-mining-network
Data is present
{'height': 57, 'width': 768}
{'height': 72, 'width': 768}
Run 1: 1M

Run 0: 7D hydranet
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 1: 1M hydranet
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 2: 3M hydranet
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 3: 1Y hydranet
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 4: YTD hydranet
Size: 5
Data is missing. Skipping currency!
Data is present
{'height': 57, 'width': 768}
{'height': 72, 'width': 768}
Run 0: 7D tranquil-finance
Data is present
{'height': 57, 'width': 768}
{'height': 72, 'width': 768}
Run 1: 1M tranquil-finance
Data is present
{'height': 57, 'width': 768}
{'height': 72, 'width': 768}
Run 2: 3M tranquil-finance
Data is present
{'height': 57, 'width': 768}
{'height': 72, 'width': 768}
Run 3: 1Y tranquil-finance
Data is present
{'height': 57, 'width': 768}
{'height': 72, 'width': 768}
Run 4: YTD tranquil-finance
Size: 5
Data is missing. Skipping currency!
Data is presen

Run 0: 7D parrot-protocol
Data is present
{'height': 57, 'width': 768}
{'height': 72, 'width': 768}
Run 1: 1M parrot-protocol
Data is present
{'height': 57, 'width': 768}
{'height': 72, 'width': 768}
Run 2: 3M parrot-protocol
Data is present
{'height': 57, 'width': 768}
{'height': 72, 'width': 768}
Run 3: 1Y parrot-protocol
Data is present
{'height': 57, 'width': 768}
{'height': 72, 'width': 768}
Run 4: YTD parrot-protocol
Size: 5
Data is missing. Skipping currency!
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 0: 7D bodrumspor-fan-token
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 1: 1M bodrumspor-fan-token
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 2: 3M bodrumspor-fan-token
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 3: 1Y bodrumspor-fan-token
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 4: YTD bodrumspor-fan-token
Siz

{'height': 48, 'width': 768}
Run 2: 3M tuzki
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 3: 1Y tuzki
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 4: YTD tuzki
Size: 5
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 0: 7D metax
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 1: 1M metax
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 2: 3M metax
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 3: 1Y metax
Data is present
{'height': 57, 'width': 768}
{'height': 48, 'width': 768}
Run 4: YTD metax
Size: 5
Data is missing. Skipping currency!
Data is missing. Skipping currency!
Data is missing. Skipping currency!
Size: 5
Data is missing. Skipping currency!
Data is present
{'height': 57, 'width': 768}
{'height': 40, 'width': 712}
{'height': 48, 'width': 768}
Run 0: 7D duzce-token
Data is present
{'height': 57,

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00EBA813+48355]
	(No symbol) [0x00E4C4B1]
	(No symbol) [0x00D55358]
	(No symbol) [0x00D3D293]
	(No symbol) [0x00D9E37B]
	(No symbol) [0x00DAC473]
	(No symbol) [0x00D9A536]
	(No symbol) [0x00D782DC]
	(No symbol) [0x00D793DD]
	GetHandleVerifier [0x0111AABD+2539405]
	GetHandleVerifier [0x0115A78F+2800735]
	GetHandleVerifier [0x0115456C+2775612]
	GetHandleVerifier [0x00F451E0+616112]
	(No symbol) [0x00E55F8C]
	(No symbol) [0x00E52328]
	(No symbol) [0x00E5240B]
	(No symbol) [0x00E44FF7]
	BaseThreadInitThunk [0x74F57D59+25]
	RtlInitializeExceptionChain [0x76FFB74B+107]
	RtlClearBits [0x76FFB6CF+191]
